<a href="https://colab.research.google.com/github/asingh-ml/NER_Product_Brand/blob/master/train_spacy_product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
! pip install spacy==2.2.3

     |████████████████████████████████| 10.4MB 3.4MB/s 
     |████████████████████████████████| 122kB 39.8MB/s 
     |████████████████████████████████| 3.7MB 26.7MB/s 
     |████████████████████████████████| 2.2MB 26.7MB/s 
  Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninstalled blis-0.2.4
  Found existing installation: thinc 7.0.8
    Uninstalling thinc-7.0.8:
      Successfully uninstalled thinc-7.0.8
  Found existing installation: spacy 2.1.9
    Uninstalling spacy-2.1.9:
      Successfully uninstalled spacy-2.1.9


In [0]:
# !cp "/content/drive/My Drive/Ner_data/data/test.csv" "data/test.csv"
import pickle
with open ('/content/drive/My Drive/Ner_data/data/traindata_spacy_v1', 'rb') as fp:
    train_data = pickle.load(fp)

In [2]:
len(train_data)

281409

In [3]:
import spacy
spacy.__version__

'2.2.3'

In [0]:
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from tqdm import tqdm

In [0]:
def train_spacy(model=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            print("Starting iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(train_data, size=compounding(100.0, 200.0, 1.001))
            for batch in tqdm(batches):
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    return nlp

In [0]:
nlp = train_spacy(n_iter=50)

Created blank 'en' model
Starting iteration 0


1603it [08:46,  3.49it/s]


Losses {'ner': 390697.55290317535}
Starting iteration 1


1603it [08:50,  3.15it/s]


Losses {'ner': 249837.8449831009}
Starting iteration 2


...

1603it [08:43,  3.41it/s]


Losses {'ner': 132729.27409422398}
Starting iteration 19


1603it [08:42,  3.39it/s]


Losses {'ner': 133322.94478428364}
Starting iteration 20


364it [01:26,  3.73it/s]

In [0]:
# test the trained model
for text, _ in train_data:
    doc = nlp(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [25]:
# save model to output directory
output_dir = Path('/content/drive/My Drive/Ner_data/data/spacy_model_v2')
if not output_dir.exists():
    output_dir.mkdir()
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to /content/drive/My Drive/Ner_data/data/spacy_model_v2


In [26]:
# test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in train_data:
    doc = nlp2(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    break


Loading from /content/drive/My Drive/Ner_data/data/spacy_model_v2
Entities [('persol', 'BRAND'), ('sunglasses', 'PRODUCT')]
Tokens [('persol', 'BRAND', 3), ('po3046s', '', 2), ('black', '', 2), ('/', '', 2), ('green', '', 2), ('sunglasses', 'PRODUCT', 3), ('49', '', 2), ('mm', '', 2)]


In [8]:
nlp2

In [27]:
text = "Microsoft Office Home and Business 2019 Download 1 Person Compatible on Windows 10 and Apple macOS"
doc = nlp2(text)
print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Tokens [('Microsoft', 'BRAND', 3), ('Office', '', 2), ('Home', '', 2), ('and', '', 2), ('Business', '', 2), ('2019', '', 2), ('Download', '', 2), ('1', '', 2), ('Person', '', 2), ('Compatible', '', 2), ('on', '', 2), ('Windows', '', 2), ('10', '', 2), ('and', '', 2), ('Apple', '', 2), ('macOS', '', 2)]


In [42]:
text = "Microsoft Office Home and Business 2019 Download 1 Person Compatible on Windows 10 and Apple macOS"
doc = nlp2(text)
print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc if t.ent_type_ == 'BRAND' or t.ent_type_ == 'PRODUCT'])

Tokens [('Microsoft', 'BRAND', 3)]
